
# Text parser for volume two

This jupyter notebook file houses all of the code used during the porject. 

## Do not run any of the scripts as doing so will override most of the output data which has been sanitised manually. Only use it as a reference, and write new code.

In [45]:
# DO NOT USE

# To remove frontmatter and backmatter from the .txt transcriptions of the PDF file

import os
with open("./input/raw.txt", "r") as infile:
    data = infile.readlines()
    
    # Usable Parts of Section Two
    with open("input/Southern.txt", 'w') as outfile:
        outfile.writelines(data[3074:18607])
        
    # Usable Parts of Section Three
    with open("Northern.txt", 'w') as outfile:
        outfile.writelines(data[18611:39874])
        
    # Usable Parts of Section Four
    with open("Jiuliyang.txt", 'w') as outfile:
        outfile.writelines(data[39727:43643])

In [1]:
# DO NOT USE
# Remove watermarks

import re


def refine(file):
    # Function to process raw data for future further operation
    
    buffer = None
    
    with open(file, "r+") as f:
        buffer = f.read()
        f.seek(0)
        
        # Delete Watermarks
        buffer = re.sub("Kenneth Dean and Zhenman ZHENG - 978-90-47-44017-8", "", buffer).rstrip()
        buffer = re.sub("Downloaded from Brill.com02/18/2020 04:31:15AM", "", buffer).rstrip()
        buffer = re.sub("via National University of Singapore - YNC Library", "", buffer).rstrip()
        
        # Delete Headers
        buffer = re.sub("the southern irrigated plain 南洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the northern irrigated plain 北洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the jiuliyang irrigated plain 九里洋水利系统", "", buffer).rstrip()
        buffer = re.sub("part (one|two|three|four|five)", "", buffer).rstrip()
        
        # Delete Page Numbers
        buffer = re.sub("\n[0-9]+\n", "", buffer).rstrip()
        buffer = re.sub("\f(.*?)\n", "", buffer).rstrip()
        buffer = re.sub("\n\n(A|V)[0-9]+\n\n\n", "", buffer).rstrip()
        
        # Delete blank lines
        buffer = re.sub("\n+", " ", buffer).rstrip()
        
        # Add line breaks at each new entry
        buffer = re.sub("((A|V)[0-9]{3})", "\n\g<1>", buffer)

        f.truncate()        
        f.write(buffer)

In [46]:
# DO NOT USE
# Remove watermarks

refine("Southern.txt")
refine("Northern.txt")
refine("Jiuliyang.txt")



In [1]:
# DO NOT USE
# Read god names and create JSON of them

import os, json

godsList= []

with open("input/list_of_gods.txt", 'r') as infile:
    data = infile.readlines()
    
    for line in data:
        EngText=" ".join(line.split()[:-1]).split("/")
        
        lineEntry = {
            "Name": EngText[0],
            "Name_zh": line.split()[-1]
        }
        
        if len(EngText) > 1:
            lineEntry["Aliases"] = EngText[1:]
        else:
            lineEntry["Aliases"] = []
        
        godsList.append(lineEntry)
        
    infile.close()

with open("./output/god_names.json", "w") as outfile:
    json.dump(godsList, outfile, ensure_ascii=False)
    outfile.close()

In [2]:
# DO NOT USE
# Load and map villages into Javascript objects for future use

import re, os, json

village_index = []

with open("./input/contents.txt") as infile:
    for line in infile:
        buffer = re.split("(V[0-9]{3})", line)      
        entry = {
            "UID_A": buffer[0][0:4],
            "Name": " ".join(buffer[0].split(" ")[1:-2]),
            "Name_zh": buffer[0].split(" ")[-2]
        }

        runs = int((len(buffer)-1)/2 + 1)
        villageList=[]

        for i in range(1,runs):
            village_entry = {
                "UID_V": buffer[i*2-1],
                "Name": buffer[i*2].split(" ")[1],
                "Name_zh": buffer[i*2].split(" ")[2]
            }
            villageList.append(village_entry)

        entry["Num_Villages"] = len(villageList)
        entry["Villages"] = villageList

        village_index.append(entry)

with open("./output/village_index.json", "w") as outfile:
    json.dump(village_index, outfile, ensure_ascii=False)
    outfile.close()

infile.close()
    
#search function
def whatAlliance(searchterm):
    for alliance in village_index:
        for village in alliance["Villages"]:
            if village["UID_V"] == searchterm:
                return alliance
        
    return -1
    

In [3]:
# DO NOT USE
# Load surname mapping functions

with open('./input/Surname_Groups.json', "r") as surnameFile:
    surnameGroups = json.load(surnameFile)
surnameFile.close()

#search function
def whatSurnames(searchterm):
    for village in surnameGroups:
        if village["UID"] == searchterm:
            return village
    return -1


In [42]:
# DO NOT USE
# Text Parsing to create a "Village_Master" master object from which data can be accessed. 
 
import json, regex

def createVillageFiles(path, irrNetwork):
    
    with open(path) as infile:
        for line in infile:
            if line[0] == "V":
                uid = line[0:4]                
                buffer1 = line.split(" ", 3)
                buffer2 = buffer1[3].partition("2. Surname groups")
                buffer3 = buffer2[2].partition("3. Village temples")
                buffer4 = buffer3[2].partition("4. Rituals")
                buffer5 = buffer4[2].partition("Birthday celebrations of the gods")
                buffer6 = buffer5[2].partition("5. Ritual Groups")
                
                if buffer6 is None:
                    buffer6 = ""
            
                # Search for Alliance details
                allianceDetails = whatAlliance(uid)
                
                temples = regex.split("(?:^|\.\s)\d+\)(?=\s[A-Z])", buffer4[0])
            
                entry = {
                    "UID": buffer1[0],
                    "Name": buffer1[1],
                    "Name_zh": buffer1[2],
                    "Alliance": allianceDetails["Name"],
                    "Alliance_UID": allianceDetails["UID_A"],
                    "Alliance_zh": allianceDetails["Name_zh"],
                    "Irrigation_Region": irrNetwork,
                    "Raw_Text": {                        
                        "Village_Settlement": buffer2[0][22:],
                        "Surname_Groups": buffer3[0][1:],
                        "Village_Temples": buffer4[0][1:],
                        "Rituals": {
                            "Yuanxiao processions": buffer5[0][23:],
                            "Birthday celebrations of the gods": buffer5[2][1:].strip(),
                            "Ritual Groups": buffer6[2]
                        }
                    }
                }
                
                # Add administrative village if applicable
                admin_village = re.search("\w* Administrative Village\s([^\.\s]+)", entry["Raw_Text"]["Village_Settlement"])
                if admin_village != None:
                    split_words = admin_village.group(0).split(" ")
                    entry["Raw_Text"]["Administrative_Village"] = " ".join(split_words[:-1])
                    entry["Raw_Text"]["Administrative_Village_zh"] = "".join(split_words[-1])                    
                else:
                    entry["Administrative_Village"] = "-1"
                    entry["Administrative_Village_zh"] = "-1"
                    
               # Find Population
                pop = re.search("[0-9,]*(?=\speople|\sresidents|\sinhabitants)", entry["Raw_Text"]["Village_Settlement"])
                
                if pop == None:
                    pop = re.search("[\w\s]*(people|residents|inhabitants|population)[\w\s]*", entry["Raw_Text"]["Village_Settlement"])
                    
                    if pop == None:
                        entry["Population"] = -1
                    else:                    
                        pop = re.search("[0-9,]*", pop.group(0))
                    
                        if pop == None:
                            entry["Population"] = -1
                        else:
                            entry["Population"] = pop.group(0)
                else:
                    entry["Population"] = pop.group(0)
                
                # Number of temples
                # Main Gods Worshipped
                Main_Gods = []
                num_temple = 0
                
                    for god in godsList:
                        if god["Name"] in templedata.group(0):
                            if not god["Name"] in Main_Gods:
                                Main_Gods.append(god["Name"])
                        else:
                            for aliases in god["Aliases"]:
                                if aliases in templedata.group(0):
                                    if not god["Name"] in Main_Gods:
                                        Main_Gods.append(god["Name"])
                                        
                    num_temple = num_temple + 1
                
                entry["Num_Temples"] = str(num_temple)
                entry["Main_Gods"] = Main_Gods
                entry["Num_Main_Gods"] = str(len(Main_Gods))
                
                # Secondary Gods Worshipped
                Sec_Gods = []
                for templedata in re.finditer("\.*(secondary)([^.]+)\.", entry["Raw_Text"]["Village_Temples"], flags=re.IGNORECASE):
                    # print(uid, templedata)
                    for god in godsList:
                        if god["Name"] in templedata.group(0):
                            if not god["Name"] in Sec_Gods:
                                Sec_Gods.append(god["Name"])
                        else:
                            for aliases in god["Aliases"]:
                                if aliases in templedata.group(0):
                                    if not god["Name"] in Sec_Gods:
                                        Sec_Gods.append(god["Name"])
                
                entry["Secondary_Gods"] = Sec_Gods
                entry["Num_Secondary_Gods"] = str(len(Sec_Gods))
                
                villages.append(entry) 
                
    infile.close()
    
    # dump to file
    with open("./output/village_master.json", "w") as outfile:
        json.dump(villages, outfile, ensure_ascii=False)
        outfile.close()
        
    return villages
              

In [55]:
# DO NOT USE
#

villages = []

overview = createVillageFiles("./input/1-100.txt", "Southern")
overview = createVillageFiles("./input/101-200.txt", "Southern")
overview = createVillageFiles("./input/201-297.txt", "Southern")
overview = createVillageFiles("./input/298-400.txt", "Northern")
overview = createVillageFiles("./input/401-500.txt", "Northern")
overview = createVillageFiles("./input/501-600.txt", "Northern")
overview = createVillageFiles("./input/601-659.txt", "Northern")
overview = createVillageFiles("./input/jiuliyang.txt", "Jiuliyang")
print("Completed")

Completed


In [23]:
# DO NOT USE
# Create summary.csv

import json, csv

with open('./output/village_master.json') as infile:
    
    indata = json.load(infile)
    
    # General data including 1) UID, 2) Name, 3) Name_zh, 4) Alliance, 5) Alliance_UID, 6) Alliance_zh, 7) Irrigation_Region
    keysToExtract = ["UID", "Name", "Name_zh", "Alliance", "Alliance_UID", "Alliance_zh", "Irrigation_Region"]
    genDataBuffer = []
        
    for entry in indata:
        
        outEntry = {}
        
        for var in entry:
            if var in keysToExtract:
                outEntry[var] = entry[var]
                
        genDataBuffer.append(outEntry)
        
    with open('./output/summary.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keysToExtract, delimiter=',')
        dict_writer.writeheader()
        dict_writer.writerows(genDataBuffer)    
    

In [24]:
# DO NOT USE
# Parse village data to a CSV which can be imported into PostGIS

import json, csv

with open('./output/village_master.json') as infile:
    
    indata = json.load(infile)
    
    villages_out= []
    
    for entry in indata:
        
        entry = {
            "UID_V": entry["UID"],
            "name": entry["Name"],
            "name_zh": entry["Name_zh"],
            "irrigation_region": entry["Irrigation_Region"],
            "population": entry["Population"],
        }
        
        villages_out.append(entry)
    
    with open("./output/Villages.csv", "w") as outfile:
        writer = csv.DictWriter(outfile, fieldnames=villages_out[0].keys())
        writer.writeheader()
        writer.writerows(villages_out)       
        

In [5]:
# DO NOT USE
# Parse village data to a file with the Raw Text of each section

import json, csv

with open('./output/village_master.json') as infile:
    indata = json.load(infile)
    
    villages_out = []
    
    for in_entry in indata:
        
        out_entry = {
            "UID": in_entry["UID"],
            "Name": in_entry["Name"],
            "Name_zh": in_entry["Name_zh"],
            "Village_Settlement": in_entry["Raw_Text"]["Village_Settlement"],
            "Surname_Groups": in_entry["Raw_Text"]["Surname_Groups"],
            "Village_Temples": in_entry["Raw_Text"]["Village_Temples"],
            "Rituals_Yuanxiao_Processions": in_entry["Raw_Text"]["Rituals"]["Yuanxiao processions"],
            "Rituals_Birthday_Celebration_of_gods": in_entry["Raw_Text"]["Rituals"]["Birthday celebrations of the gods"],
            "Rituals_Ritual_Groups": in_entry["Raw_Text"]["Rituals"]["Ritual Groups"]
        }
        
        villages_out.append(out_entry)
    
    with open("./output/Raw_Data.csv", "w") as outfile:
        writer = csv.DictWriter(outfile, fieldnames=villages_out[0].keys(),quoting=csv.QUOTE_ALL, escapechar='\\')
        writer.writeheader()
        writer.writerows(villages_out)    
        
    with open("./output/Raw_Data.json", "w") as outfile:
        json.dump(villages_out, outfile)

In [26]:
# DO NOT USE
# Parse village data to a CSV which can be imported into PostGIS

import json, csv

with open('./output/god_names.json') as infile:
    indata = json.load(infile)
    
    data_out = []
    
    for in_entry in indata:
        
        alias_string = "{" + ', '.join(in_entry["Aliases"]) + "}"
        
        print(alias_string)
        
        out_entry = {
            "God_Name": in_entry["Name"],
            "God_Name_zh": in_entry["Name_zh"],
            "Aliases": alias_string
        }
        
        print(out_entry)
        
        data_out.append(out_entry)
    
    with open("./output/God_Aliases.csv", "w") as outfile:
        writer = csv.DictWriter(outfile, fieldnames=data_out[0].keys(),quoting=csv.QUOTE_ALL, escapechar='\\')
        writer.writeheader()
        writer.writerows(data_out) 

{}
{'God_Name': '18 luo han', 'God_Name_zh': '18罗汉', 'Aliases': '{}'}
{Asi}
{'God_Name': 'Asan', 'God_Name_zh': '阿三阿四', 'Aliases': '{Asi}'}
{}
{'God_Name': 'Aye gong', 'God_Name_zh': '阿爷公', 'Aliases': '{}'}
{}
{'God_Name': 'Anling shenggong', 'God_Name_zh': '安灵圣公', 'Aliases': '{}'}
{}
{'God_Name': 'Anshan shengwang', 'God_Name_zh': '安善圣王', 'Aliases': '{}'}
{}
{'God_Name': 'Ansheng lingfei', 'God_Name_zh': '安圣灵妃', 'Aliases': '{}'}
{}
{'God_Name': 'Aotou ma', 'God_Name_zh': '鳌头妈', 'Aliases': '{}'}
{}
{'God_Name': 'Baban', 'God_Name_zh': '八班', 'Aliases': '{}'}
{}
{'God_Name': 'Babaoye', 'God_Name_zh': '八宝爷', 'Aliases': '{}'}
{}
{'God_Name': 'Bajiang', 'God_Name_zh': '八将', 'Aliases': '{}'}
{}
{'God_Name': 'Baxian', 'God_Name_zh': '八仙', 'Aliases': '{}'}
{}
{'God_Name': 'Bai daren', 'God_Name_zh': '白大人', 'Aliases': '{}'}
{}
{'God_Name': 'Baifu daren', 'God_Name_zh': '白府大人', 'Aliases': '{}'}
{}
{'God_Name': 'Baigong daren', 'God_Name_zh': '白公大人', 'Aliases': '{}'}
{}
{'God_Name': 'Baihu jiangj

In [27]:
# DO NOT USE
# Parse village data to a CSV which can be imported into PostGIS

import json, csv

with open('./output/village_index.json') as infile:
    
    indata = json.load(infile)
    data_out = []
    for Alliance in indata:
        for Village in Alliance["Villages"]:
            data_out.append({
                "UID_V": Village["UID_V"],
                "UID_A": Alliance["UID_A"]
            })
    
    with open("./output/alliance_membership.csv", "w") as outfile:
        writer = csv.DictWriter(outfile, fieldnames=data_out[0].keys(),quoting=csv.QUOTE_ALL, escapechar='\\')
        writer.writeheader()
        writer.writerows(data_out)
    

In [28]:
# DO NOT USE
# Parse village data to a CSV which can be imported into PostGIS

import json, csv

with open('./output/village_master.json') as infile:
    
    indata = json.load(infile)
    data_out = []
    
    for Village in indata:
        data_out.append({
            "UID_V": Village["UID"],
            "Population": Village["Population"]
        })
        
    print(data_out)
    
    with open("./output/pop.csv", "w") as outfile:
        writer = csv.DictWriter(outfile, fieldnames=data_out[0].keys(), escapechar='\\')
        writer.writeheader()
        writer.writerows(data_out)
    

[{'UID_V': 'V001', 'Population': '800'}, {'UID_V': 'V002', 'Population': '150'}, {'UID_V': 'V003', 'Population': '200'}, {'UID_V': 'V004', 'Population': '800'}, {'UID_V': 'V005', 'Population': '210'}, {'UID_V': 'V006', 'Population': '1000'}, {'UID_V': 'V007', 'Population': '800'}, {'UID_V': 'V008', 'Population': '2400'}, {'UID_V': 'V009', 'Population': '1500'}, {'UID_V': 'V010', 'Population': '500'}, {'UID_V': 'V011', 'Population': '1300'}, {'UID_V': 'V012', 'Population': '500'}, {'UID_V': 'V013', 'Population': '655'}, {'UID_V': 'V014', 'Population': '509'}, {'UID_V': 'V015', 'Population': '600'}, {'UID_V': 'V016', 'Population': '500'}, {'UID_V': 'V017', 'Population': '500'}, {'UID_V': 'V018', 'Population': '1000'}, {'UID_V': 'V019', 'Population': '600'}, {'UID_V': 'V020', 'Population': '800'}, {'UID_V': 'V021', 'Population': '400'}, {'UID_V': 'V022', 'Population': '300'}, {'UID_V': 'V023', 'Population': '100'}, {'UID_V': 'V024', 'Population': '2400'}, {'UID_V': 'V025', 'Population': '

In [35]:
# DO NOT USE
# Process text of "Village_Temple" entries for each village, extracting major gods and temple names, etc.

import json, csv, re

with open('./output/village_master.json') as infile:
    
    indata = json.load(infile)
    data_out = []
    primaryGod_out = []
    secondaryGod_out = []

    with open('./output/god_names.json') as godfile:
    
        goddata = json.load(godfile)
    
    
        for v_entry in indata:
            templeNo = 0
            temple_entry = v_entry["Raw_Text"]["Village_Temples"]
            temples_array = re.split("(?:^|\.\s|\)\s)\d+\)(?=\s[A-Z])", temple_entry)
            for t in temples_array:
                
                
                if t == "":
                    continue
                else:
                    
                    templeNo += 1
                    
                     # identify primary and secondary
                    foo = re.split("Secondary", t)
                    primarygods =[]
                    secondarygods = []
                    
                    primaryBlock = foo[0]
                    for godEntry in goddata:
                        if godEntry["Name"] in primaryBlock:
                            if not godEntry["Name"] in primarygods:
                                primarygods.append(godEntry["Name"])
                        else:
                            for aliases in godEntry["Aliases"]:
                                if aliases in primaryBlock:
                                    if not godEntry["Name"] in primarygods:
                                        primarygods.append(godEntry["Name"]) 
                                                            
                    if len(foo) == 2:
                        secondaryblock = foo[1]
                        for godEntry in goddata:
                            if godEntry["Name"] in secondaryblock:
                                if not godEntry["Name"] in secondarygods:
                                    secondarygods.append(godEntry["Name"])
                            else:
                                for aliases in godEntry["Aliases"]:
                                    if aliases in secondaryblock:
                                        if not godEntry["Name"] in secondarygods:
                                            secondarygods.append(godEntry["Name"])
                    
                    firstHanzi = re.search("([\u4E00-\u9FCC\u3400-\u4DB5\uFA0E\uFA0F\uFA11\uFA13\uFA14\uFA1F\uFA21\uFA23\uFA24\uFA27-\uFA29]|[\ud840-\ud868][\udc00-\udfff]|\ud869[\udc00-\uded6\udf00-\udfff]|[\ud86a-\ud86c][\udc00-\udfff]|\ud86d[\udc00-\udf34\udf40-\udfff]|\ud86e[\udc00-\udc1d])+", t)
                    uid_t = v_entry["UID"] + "-" + str(templeNo)
                    
                    buffer = {
                        "uid_t": uid_t,
                        "name": t[0:firstHanzi.span(0)[0]].strip(),
                        "name_zh": firstHanzi.group(0),
                        "village": v_entry["UID"],
                        "raw_Text": t
                    }
                    
                    data_out.append(buffer)
                    
                    for god_entry in primarygods:
                        buffer2 = {
                            "uid_t": uid_t,
                            "god_name":god_entry
                        }
                        
                        primaryGod_out.append(buffer2)
                        
                    for god_entry in secondarygods:
                        buffer2 = {
                            "uid_t": uid_t,
                            "god_name":god_entry
                        }
                        
                        secondaryGod_out.append(buffer2)
                        

with open("./output/secondary_gods.csv", "w") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=secondaryGod_out[0].keys(), escapechar='\\')
    writer.writeheader()
    writer.writerows(secondaryGod_out)

with open("./output/primary_gods.csv", "w") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=primaryGod_out[0].keys(), escapechar='\\')
    writer.writeheader()
    writer.writerows(primaryGod_out)
                    
with open("./output/temples.csv", "w") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=data_out[0].keys(), escapechar='\\')
    writer.writeheader()
    writer.writerows(data_out)
    

In [19]:
# DO NOT USE
# Process text of "Yuanxiao processions" entries for each village, extracting major gods and temple names, etc.

import json, csv, re
from nltk.tokenize import sent_tokenize

with open('./output/village_master.json') as infile:
    indata = json.load(infile)
    ritualsout=[]
    
    with open('./output/god_names.json') as godfile:
        goddata = json.load(godfile)
    
        for entry in indata:
            entry_text = entry["Raw_Text"]["Rituals"]["Yuanxiao processions"]

            sentences = sent_tokenize(entry_text)
            buffer = ""
            array = []

            for sentence in sentences:
                match = re.search("(\d{1,2}\/\d{1,2}[\-\–]\d{1,2}|\d{1,2}\/\d{1,2}[\-\–]\d{1,2}\/\d{1,2}|\d{1,2}\/\d{1,2})", sentence)

                if match != None or re.search("lunar month",sentence):
                    if len(buffer) == 0:
                        array.append(buffer)
                        buffer = sentence
                    else:
                        array.append(buffer)
                        buffer = sentence
                else:
                    buffer = buffer + " " + sentence

            array.append(buffer)

            array = array[1:]

            entryno = 0

            for e in array:

                entryno = entryno + 1
                startdate = "MM-DD"
                enddate = "MM-DD"

                date = re.search("(\d{1,2}\/\d{1,2}[\-\–]\d{1,2}|\d{1,2}\/\d{1,2}[\-\–]\d{1,2}\/\d{1,2}|\d{1,2}\/\d{1,2})", e)
                if date != None:
                    if date.group(0).count("–") == 0:
                        alpha = re.split("/",date.group(0))
                        if len(alpha[0]) == 1:
                            alpha[0] = "0" + alpha[0]
                        if len(alpha[1]) == 1:
                            alpha[1] = "0" + alpha[1]

                        startdate = alpha[0] + "-" + alpha[1]
                        enddate = startdate

                    if date.group(0).count("/") == 2:
                        alpha = re.split("–",date.group(0))
                        bravo = re.split("/",alpha[0])
                        charlie = re.split("/",alpha[1])

                        if len(bravo[0]) == 1:
                            bravo[0] = "0" + bravo[0]
                        if len(bravo[1]) == 1:
                            bravo[1] = "0" + bravo[1]
                        if len(charlie[0]) == 1:
                            charlie[0] = "0" + charlie[0]
                        if len(charlie[1]) == 1:
                            charlie[1] = "0" + charlie[1]

                        startdate = bravo[0] + "-" + bravo[1]
                        enddate = charlie[0] + "-" + charlie[1]

                    if date.group(0).count("–") == 1 and date.group(0).count("/") == 1:
                        alpha = re.split("–",date.group(0))
                        bravo = re.split("/",alpha[0])

                        if len(bravo[0]) == 1:
                            bravo[0] = "0" + bravo[0]
                        if len(bravo[1]) == 1:
                            bravo[1] = "0" + bravo[1]
                        if len(alpha[0]) == 1:
                            alpha[0] = "0" + alpha[0]

                        startdate = bravo[0] + "-" + bravo[1]
                        enddate = bravo[0] + "-" + alpha[1]

                    gods_involved = []
                    for godEntry in goddata:

                        if godEntry["Name"] in e:
                            if not godEntry["Name"] in gods_involved:
                                gods_involved.append(godEntry["Name"])
                        else:
                            for aliases in godEntry["Aliases"]:
                                if aliases in gods_involved:
                                    if not godEntry["Name"] in gods_involved:
                                        gods_involved.append(godEntry["Name"])
                            for aliases in godEntry["Name_zh"]:
                                if aliases in gods_involved:
                                    if not godEntry["Name"] in gods_involved:
                                        gods_involved.append(godEntry["Name"])

                ritualsout.append({
                    "startdate": startdate,
                    "enddate": enddate,
                    "uid_r": entry["UID"]+"-"+str(entryno),
                    "text": e,
                    "gods_involved": gods_involved
                })

with open('./output/processions.json', 'w', encoding='utf8') as outfile:
    json.dump(ritualsout, outfile, ensure_ascii=False)
    
with open("./output/processions.csv", "w") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=ritualsout[0].keys(), escapechar='\\')
    writer.writeheader()
    writer.writerows(ritualsout)

In [22]:
# DO NOT USE
# Process text of "Birthday Celerbations of the gods" entries for each village, extracting major gods and temple names, etc.

import json, csv, re
from nltk.tokenize import sent_tokenize

with open('./output/village_master.json') as infile:
    indata = json.load(infile)
    ritualsout=[]
    
    with open('./output/god_names.json') as godfile:
        goddata = json.load(godfile)
    
        for entry in indata:
            entry_text = entry["Raw_Text"]["Rituals"]["Birthday celebrations of the gods"]

            sentences = sent_tokenize(entry_text)
            buffer = ""
            array = []

            for sentence in sentences:
                match = re.search("(\d{1,2}\/\d{1,2}[\-\–]\d{1,2}|\d{1,2}\/\d{1,2}[\-\–]\d{1,2}\/\d{1,2}|\d{1,2}\/\d{1,2})", sentence)

                if match != None or re.search("lunar month",sentence):
                    if len(buffer) == 0:
                        array.append(buffer)
                        buffer = sentence
                    else:
                        array.append(buffer)
                        buffer = sentence
                else:
                    buffer = buffer + " " + sentence

            array.append(buffer)

            array = array[1:]

            entryno = 0

            for e in array:

                entryno = entryno + 1
                startdate = "MM-DD"
                enddate = "MM-DD"

                date = re.search("(\d{1,2}\/\d{1,2}[\-\–]\d{1,2}|\d{1,2}\/\d{1,2}[\-\–]\d{1,2}\/\d{1,2}|\d{1,2}\/\d{1,2})", e)
                if date != None:
                    if date.group(0).count("–") == 0:
                        alpha = re.split("/",date.group(0))
                        if len(alpha[0]) == 1:
                            alpha[0] = "0" + alpha[0]
                        if len(alpha[1]) == 1:
                            alpha[1] = "0" + alpha[1]

                        startdate = alpha[0] + "-" + alpha[1]
                        enddate = startdate

                    if date.group(0).count("/") == 2:
                        alpha = re.split("–",date.group(0))
                        bravo = re.split("/",alpha[0])
                        charlie = re.split("/",alpha[1])

                        if len(bravo[0]) == 1:
                            bravo[0] = "0" + bravo[0]
                        if len(bravo[1]) == 1:
                            bravo[1] = "0" + bravo[1]
                        if len(charlie[0]) == 1:
                            charlie[0] = "0" + charlie[0]
                        if len(charlie[1]) == 1:
                            charlie[1] = "0" + charlie[1]

                        startdate = bravo[0] + "-" + bravo[1]
                        enddate = charlie[0] + "-" + charlie[1]

                    if date.group(0).count("–") == 1 and date.group(0).count("/") == 1:
                        alpha = re.split("–",date.group(0))
                        bravo = re.split("/",alpha[0])

                        if len(bravo[0]) == 1:
                            bravo[0] = "0" + bravo[0]
                        if len(bravo[1]) == 1:
                            bravo[1] = "0" + bravo[1]
                        if len(alpha[0]) == 1:
                            alpha[0] = "0" + alpha[0]

                        startdate = bravo[0] + "-" + bravo[1]
                        enddate = bravo[0] + "-" + alpha[1]

                    gods_involved = []
                    for godEntry in goddata:

                        if godEntry["Name"] in e:
                            if not godEntry["Name"] in gods_involved:
                                gods_involved.append(godEntry["Name"])
                        else:
                            for aliases in godEntry["Aliases"]:
                                if aliases in gods_involved:
                                    if not godEntry["Name"] in gods_involved:
                                        gods_involved.append(godEntry["Name"])
                            for aliases in godEntry["Name_zh"]:
                                if aliases in gods_involved:
                                    if not godEntry["Name"] in gods_involved:
                                        gods_involved.append(godEntry["Name"])

                ritualsout.append({
                    "startdate": startdate,
                    "enddate": enddate,
                    "uid_r": entry["UID"]+"-"+str(entryno),
                    "text": e,
                    "gods_involved": gods_involved
                })

with open('./output/birthday_celebrations.json', 'w', encoding='utf8') as outfile:
    json.dump(ritualsout, outfile, ensure_ascii=False)
    
with open("./output/birthday_celebrations.csv", "w") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=ritualsout[0].keys(), escapechar='\\')
    writer.writeheader()
    writer.writerows(ritualsout)
    
    

In [21]:
# DO NOT USE
# Proces data in a way friendly for Select2

import json, csv

output = []
    
with open('./output/Villages.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None)
    
    
    for row in spamreader:
        output.append({
            "id": row[0],
            "title": row[0] + ": "+ row[1] + " " + row[2]
        })

with open("./output/village_search.json", "w") as outfile:
    json.dump(output, outfile, ensure_ascii=False)
    outfile.close()
    
with open("./output/god_names.json") as godfile:
    goddata = json.load(godfile)
    
with open('./output/primary_gods.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None)
    
    output = []
    
    gods_buffer = {}
    
    for row in spamreader:
        if row[1] in gods_buffer:
            gods_buffer[row[1]].append(row[0][0:4])
        else: 
            gods_buffer[row[1]] = [row[0][0:4]]
                
                
    for entry in sorted(gods_buffer):
        
        name_zh = ""
        
        for god_name_entry in goddata:
            
            if god_name_entry["Name"] == entry: 
                name_zh = god_name_entry["Name_zh"]
                
        if name_zh == "":
            raise Exception("Chinese name not found for ", entry)
        
        output.append({
            "id": gods_buffer[entry],
            "title": entry + " " + name_zh
        })
        
with open("./output/primary_god_search.json", "w") as outfile:
    json.dump(output, outfile, ensure_ascii=False)

    outfile.close()

In [5]:
# DO NOT USE
# Process text for the calendar function

import json, csv, re

output = []
    
with open('./output/processions.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(spamreader, None)
    
    output=[]

    # Create array
    for i in range(1,13):
        month = str(i)
        if len(month) == 1:
            month="0"+month
        
        for i in range(1,32):
            day = str(i)
            if len(day) == 1:
                day="0"+day
            date = month + "-" + day

            output.append({
                "date": date,
                "ID": [],
                "weight": 0,
                "gods": {}
            })

    for entry in spamreader:
        
        # one-day event?
        if entry[0] == entry[1]:
            
            ## Filter out entries without dates
            if entry[0] == "MM-DD":
                continue;
            else:
                correct_entry = list(filter(lambda k: k["date"] == entry[0], output))[0]
                
            correct_entry["weight"] = correct_entry["weight"] + 1
            
        else:
            start_date = re.split("-", entry[0])
            m1 = int(start_date[0])
            d1 = int(start_date[1])
            
            end_date = re.split("-", entry[1])
            m2 = int(end_date[0])
            d2 = int(end_date[1])
            
            duration = 31 * (m2-m1) - d1 + d2 + 1
            
            if m1 == m2:
                if (len(str(m1))) == 1:
                    m1 = "0" + str(m1)
                
                for i in range (d1, d2+1):
                    d = str(i)
                    if (len(str(i))) == 1:
                        d = "0" + str(i)
                    
                    correct_entry = list(filter(lambda k: k["date"] == str(m1)+"-"+d, output))[0]
                    
                    correct_entry["weight"] = correct_entry["weight"] + (1/duration)
                    
                    correct_entry["ID"].append(entry[2])
                    
with open("./output/test.json", "w") as outfile:
    json.dump(output, outfile, ensure_ascii=False)

    outfile.close()

In [13]:

# DO NOT USE
# Process text for the calendar function

import json, csv, re

output = []

with open("./output/god_names.json") as godfile:
    goddata = json.load(godfile)
    
with open('./output/birthday_celebrations.json', newline='') as csvfile:
    spamreader = json.load(csvfile)

    output=[]

    # Create array
    for i in range(1,13):
        month = str(i)
        if len(month) == 1:
            month="0"+month
        
        for i in range(1,32):
            day = str(i)
            if len(day) == 1:
                day="0"+day
            date = month + "-" + day

            output.append({
                "date": date,
                "ID": [],
                "weight": 0,
                "gods": {}
            })

    for entry in spamreader:
        # one-day event?
        if entry[0] == entry[1]:
            
            ## Filter out entries without dates
            if entry[0] == "MM-DD":
                continue;
            else:
                correct_entry = list(filter(lambda k: k["date"] == entry[0], output))[0]
                
            correct_entry["weight"] = correct_entry["weight"] + 1
            
        else:
            start_date = re.split("-", entry[0])
            m1 = int(start_date[0])
            d1 = int(start_date[1])
            
            end_date = re.split("-", entry[1])
            m2 = int(end_date[0])
            d2 = int(end_date[1])
            
            duration = 31 * (m2-m1) - d1 + d2 + 1
            
            if m1 == m2:
                if (len(str(m1))) == 1:
                    m1 = "0" + str(m1)
                
                for i in range (d1, d2+1):
                    d = str(i)
                    if (len(str(i))) == 1:
                        d = "0" + str(i)
                    
                    correct_entry = list(filter(lambda k: k["date"] == str(m1)+"-"+d, output))[0]
                    
                    correct_entry["weight"] = correct_entry["weight"] + (1/duration)
                    
                    correct_entry["ID"].append(entry[2])
            
                
with open("./output/birthday_sorted.json", "w") as outfile:
    json.dump(output, outfile, ensure_ascii=False)

    outfile.close()

['02-25', '02-25', 'V001-1', '2/25: The birthday celebration of Guangping Zhouwang 广平周王.', '[]']
['03-13', '03-13', 'V001-2', '3/13: The birthday celebration of Qianshi shengfei 钱氏圣妃.', "['Qianshi shengfei']"]
['07-07', '07-07', 'V001-3', '7/7: The birthday celebration of Taishan Kongwang 泰山孔王.', '[]']
['08-14', '08-14', 'V001-4', '8/14: The birthday celebration of Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. On these days opera is performed, and Daoist ritual masters are invited to conduct rites. The village organizes the event and funds are collected on a per capita basis.', "['Houtu furen', 'Zunzhu mingwang']"]
['08-15', '08-15', 'V002-1', '8/15: The birthday celebration of Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. Opera is performed, and Daoist ritual masters are invited to conduct rites. The village organizes the event and funds are collected on a per capita basis.', "['Houtu furen', 'Zunzhu mingwang']"]
['06-24', '06-24', 'V003-1', '6/24: The birthday celebration of Yushu dadi 玉枢大

['09-05', '09-05', 'V334-5', '9/5: The birthday celebration of Puji yiyonghou 普济义勇侯 is organized by the Fuyingmiao 孚应庙, and funding is raised through voluntary contributions. On most of these occasions, opera is performed and Daoist ritual masters are invited to conduct rituals. The entire village or a specific temple and its community organizes the ritual together, and funds are collected on a per capita basis.', '[]']
['02-25', '02-25', 'V335-1', '2/25: The birthday celebration of Qitian dasheng 齐天大圣.', "['Qitian dasheng']"]
['04-01', '04-01', 'V335-2', '4/1: The birthday celebration of Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人.', "['Houtu furen', 'Zunzhu mingwang']"]
['05-13', '05-13', 'V335-3', '5/13: The birthday celebration of Sima shengwang 司马圣王.', "['Sima shengwang']"]
['06-19', '06-19', 'V335-4', '6/19: The birthday celebration of Guanyin dashi 观音大士.', "['Guanyin', 'Guanyin dashi']"]
['07-23', '07-23', 'V335-5', '7/23: The birthday celebration of Lufu yima 鲁府姨妈 is organized by

['08-22', '08-22', 'V653-10', '8/22: The celebration of Puji shenghou 普济圣侯 is organized by Tianxia 田下.', "['Puji shenghou']"]
['08-23', '08-23', 'V653-11', '8/23: The celebration of Haotian dizi 昊天帝子 (Tiangong yuanshuai 田公 元帅) is organized by Qianting 前亭.', "['Haotian dizi', 'Tiangong', 'Tiangong yuanshuai']"]
['09-15', '09-15', 'V653-12', '9/15: The celebration of Heshi jiuxian 何氏九仙 is organized by Guangfuci 广福祠.', "['Heshi jiuxian']"]
['09-28', '09-28', 'V653-13', '9/28: The celebration of Mayang zungong 马洋尊公 is organized by Dongcuo 东厝.', '[]']
['11-01', '11-01', 'V653-14', '11/1: The celebration of Zhugong yuanshuai 朱公元帅 is organized by Shangxing 上兴. On these occasions, unless otherwise specified opera is performed for two days. These events are organized by the village as a whole, or by specific temples and their communities, and funding is raised on a per capita basis.', '[]']
['04-13', '04-13', 'V654-1', '4/13: The celebration of Qincha Tian daren 钦差田大人 is held.', '[]']
['05-05',

In [12]:

# DO NOT USE
# Process text for the alliance search function

import json, csv, re

output = []

with open("./output/Alliance_raw_text.json") as infile:
    indata = json.load(infile)
    
    for element in indata:
        outputext = element["UID_A"] + ": " + element["name"] + " " + element["name_zh"]
        
        output.append({
            "id": element["UID_A"],
            "text": outputext
        })
        
with open("./output/Alliance_search.json", "w") as outfile:
    json.dump(output, outfile, ensure_ascii=False)

    outfile.close()

In [83]:
# DO NOT USE
# Process text for the calendar function

import json, csv, re

output = []
    
with open("./output/birthday_celebrations.json") as infile:
    indata = json.load(infile)
    
    output=[]

    # Create array
    for i in range(1,13):
        month = str(i)
        if len(month) == 1:
            month="0"+month
        
        for i in range(1,32):
            day = str(i)
            if len(day) == 1:
                day="0"+day
            date = month + "-" + day

            output.append({
                "date": date,
                "ID": [],
                "weight": 0
            })
    
    for entry in indata:
        
           # one-day event?
        if entry["startdate"] == entry["enddate"]:
            
            ## Filter out entries without dates
            if entry["startdate"] == "MM-DD":
                continue;
            else:
                correct_entry = list(filter(lambda k: k["date"] == entry["startdate"], output))[0]
                
            correct_entry["weight"] = correct_entry["weight"] + 1
            correct_entry["ID"].append(entry["uid_r"])
            
        else:
            start_date = re.split("-", entry["startdate"])
            m1 = int(start_date[0])
            d1 = int(start_date[1])
            
            end_date = re.split("-", entry["enddate"])
            m2 = int(end_date[0])
            d2 = int(end_date[1])
            
            duration = 31 * (m2-m1) - d1 + d2 + 1
            
            if m1 == m2:
                if (len(str(m1))) == 1:
                    m1 = "0" + str(m1)
                
                for i in range (d1, d2+1):
                    d = str(i)
                    if (len(str(i))) == 1:
                        d = "0" + str(i)
                    
                    correct_entry = list(filter(lambda k: k["date"] == str(m1)+"-"+d, output))[0]
                    
                    correct_entry["weight"] = correct_entry["weight"] + (1/duration)
                    
                    correct_entry["ID"].append(entry["uid_r"])
        
        
with open("./output/birthdays_cal.json", "w") as outfile:
    json.dump(output, outfile, ensure_ascii=False)

    outfile.close()

In [1]:
# Loading surname data
import json, csv, re

output = []
    
with open("./output/birthdays_cal.json") as infile:
    indata = json.load(infile)
    

[{'month': '1', 'date': '01-01', 'ID': ['V542-1', 'V543-1'], 'weight': 0.6666666666666666}, {'month': '1', 'date': '01-02', 'ID': ['V542-1', 'V543-1'], 'weight': 0.6666666666666666}, {'month': '1', 'date': '01-03', 'ID': ['V542-1', 'V543-1', 'V558-1'], 'weight': 1.6666666666666665}, {'month': '1', 'date': '01-04', 'ID': ['V311-1', 'V679-9'], 'weight': 1.5}, {'month': '1', 'date': '01-05', 'ID': ['V179-1', 'V311-1', 'V391-1', 'V577-1', 'V690-1'], 'weight': 4.5}, {'month': '1', 'date': '01-06', 'ID': ['V632-1', 'V636-1', 'V637-1', 'V638-1', 'V639-1', 'V640-1', 'V641-1', 'V691-1'], 'weight': 8}, {'month': '1', 'date': '01-07', 'ID': ['V139-1', 'V415-1', 'V576-1', 'V584-1'], 'weight': 3.0}, {'month': '1', 'date': '01-08', 'ID': ['V477-1', 'V486-1', 'V576-1', 'V584-1'], 'weight': 3.0}, {'month': '1', 'date': '01-09', 'ID': ['V004-1', 'V132-1', 'V134-1', 'V144-1', 'V163-1', 'V171-1', 'V317-1', 'V338-1', 'V422-1', 'V470-1', 'V551-1', 'V598-1', 'V606-1', 'V621-1', 'V629-1', 'V632-2', 'V650-1',